In [4]:
!pip uninstall torchvision torchaudio -y

!pip install torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp312-cp312-linux_x86_64.whl (7.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp312-cp312-linux_x86_64.whl (3.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp312-cp312-linux_x86_64.whl (780.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached https://download.pytorch.org/whl/cu12

In [7]:
!pip uninstall torch torchvision torchaudio unsloth unsloth_zoo -y
!pip uninstall torch torchvision torchaudio unsloth unsloth_zoo -y

!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Found existing installation: unsloth 2025.11.6
Uninstalling unsloth-2025.11.6:
  Successfully uninstalled unsloth-2025.11.6
Found existing installation: unsloth_zoo 2025.11.6
Uninstalling unsloth_zoo-2025.11.6:
  Successfully uninstalled unsloth_zoo-2025.11.6
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 47.6 MB/s  0:00:07:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 111.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 139.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip uninstall torchao -y

!pip install "unsloth @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: torchao 0.14.1
Uninstalling torchao-0.14.1:
  Successfully uninstalled torchao-0.14.1
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-lmzzg28p/unsloth_ab3afff1c18c42bcbb0312480443442c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-lmzzg28p/unsloth_ab3afff1c18c42bcbb0312480443442c

  Resolved https://github.com/unslothai/unsloth.git to commit 6789c279d578278aca4af22f4ca31fc42829c9a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import torch


max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)


alpaca_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a specialized agent. You MUST output functions in the following TOON format:

name: function_name
arguments:
  arg_name: value

Do not output JSON. Do not output conversational text. Output ONLY the function block.<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""
def formatting_prompts_func(examples):
    texts = [alpaca_prompt.format(inst, out) for inst, out in zip(examples["instruction"], examples["output"])]
    return { "text" : texts }

dataset = load_dataset("csv", data_files="toon_training_data.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        output_dir = "outputs",
    ),
)

trainer.train()

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/23761 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/23761 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23,761 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,3.933100
2,3.654400
3,3.578500
4,3.338800
5,2.832600
6,2.270000
7,1.828100
8,1.366000
9,0.950300
10,0.722500


TrainOutput(global_step=60, training_loss=0.7262372488776843, metrics={'train_runtime': 232.4159, 'train_samples_per_second': 2.065, 'train_steps_per_second': 0.258, 'total_flos': 2541362420809728.0, 'train_loss': 0.7262372488776843, 'epoch': 0.02020031983839744})

In [17]:
FastLanguageModel.for_inference(model)

questions = [
    "Calculate the mortgage for a loan of $450,000 at 3.5% interest for 15 years.",
    "I need a random joke to cheer me up.",
    "Get me the news headlines for Japan."
]

print("---  Model Testing  ---")

for q in questions:
    prompt = alpaca_prompt.format(q, "")
    
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens = 128, 
        use_cache = True,
        pad_token_id = tokenizer.eos_token_id
    )
    
    result = tokenizer.batch_decode(outputs)[0]
    
    response = result.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
    response = response.replace("<|eot_id|>", "")

    print(f"\n USER: {q}")
    print(f" AGENT ACTION:\n{response}")
    print("-" * 40)

---  Model Testing  ---

 USER: Calculate the mortgage for a loan of $450,000 at 3.5% interest for 15 years.
 AGENT ACTION:
name: calculate_mortgage
arguments:
  loan_amount: 450000
  interest_rate: 3.5
  loan_term: 15
----------------------------------------

 USER: I need a random joke to cheer me up.
 AGENT ACTION:
name: get_random_joke
arguments:
  topic: general
----------------------------------------

 USER: Get me the news headlines for Japan.
 AGENT ACTION:
name: get_news_headlines
arguments:
  country: Japan
  limit: 5
----------------------------------------
